In [1]:
# import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data import MetadataCatalog
from detectron2.checkpoint import DetectionCheckpointer
import os
try: 
    from detectron2.projects.PointRend.point_rend import ColorAugSSDTransform, add_pointrend_config
except:
    from detectron2.projects.point_rend import ColorAugSSDTransform, add_pointrend_config

# Load the configuration
cfg = get_cfg()
add_pointrend_config(cfg)
cfg.merge_from_file("./model/configs/semFPN_pointrend.yaml")

cfg.MODEL.WEIGHTS = "./pretrained_models/weights/model_final_cf6ac1.pkl"  # Path to your trained weights
cfg.MODEL.RESNETS.DEPTH = 101
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 19
cfg.MODEL.POINT_HEAD.NUM_CLASSES = 19
cfg.MODEL.POINT_HEAD.TRAIN_NUM_POINTS = 2048
cfg.MODEL.POINT_HEAD.SUBDIVISION_NUM_POINTS = 8192

# Initialize the detectron2 model
detectron2_model = DefaultTrainer.build_model(cfg)

# Load weights
checkpointer = DetectionCheckpointer(detectron2_model)
checkpointer.load(cfg.MODEL.WEIGHTS)

# Extract state dict
detectron2_state_dict = detectron2_model.state_dict()

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


In [1]:
from model.pointrend import PointRendSemanticFPN

model = PointRendSemanticFPN(num_classes=1)

c:\Users\Sebastian\miniconda3\envs\thesis\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Sebastian\miniconda3\envs\thesis\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# from utils import transfer_weights

# Perform the transfer
# model = transfer_weights(detectron2_state_dict, model)

# # Save the PyTorch model weights if needed
# torch.save(model.state_dict(), "model/weights/semanticFPN_PointRend.pth")

In [2]:
import torch

model.load_state_dict(torch.load('pretrained_models\weights\semanticFPN_PointRend.pth', weights_only=True))

print(model)

RuntimeError: Error(s) in loading state_dict for PointRendSemanticFPN:
	size mismatch for point_head.fc1.weight: copying a param with shape torch.Size([256, 275, 1]) from checkpoint, the shape in current model is torch.Size([256, 257, 1]).

In [4]:
import torch
from PIL import Image
import torchvision.transforms as T

def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = T.Compose([
        T.Resize((512, 512)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image).unsqueeze(0)  

def test_model(model, input_tensor):
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
    return output

# Path to the image
image_path = 'data/segmentation_dataset/subfolder_1/images/csr0129a_front_view.png'

# Load and preprocess the image
input_tensor = load_image(image_path)

In [5]:
import matplotlib.pyplot as plt

# Load and preprocess the image
input_tensor = load_image(image_path)

# Perform inference
output = test_model(model, input_tensor)

display(output)

# Convert the output to a NumPy array and visualize it
output_mask = output.squeeze().byte().cpu().numpy()

plt.imshow(output_mask, cmap='gray')
plt.title("Segmentation Output")
plt.show()

IndexError: index 1 is out of bounds for dimension 1 with size 1

In [15]:
input_sizes = [(1, 3, 224, 448), (1, 3, 512, 512), (1, 3, 128, 128)]
for input_size in input_sizes:
    sample_input = torch.randn(*input_size)
    output = model(sample_input)
    print(f"Input size: {input_size}, Output shape: {output.shape}")

Input size: (1, 3, 224, 448), Output shape: torch.Size([1, 1, 56, 112])
Input size: (1, 3, 512, 512), Output shape: torch.Size([1, 1, 128, 128])
Input size: (1, 3, 128, 128), Output shape: torch.Size([1, 1, 32, 32])
